In [ ]:
#PROGRAM - Cell tracking group Assignment COMP9571 UNSW
#
#PROGRAMMERS - John Cawood (z5117605)
#
#LAST MODIFIED DATE - 20/07/2020
#
#USAGE - enter the folder pathway when prompted that contains the cell images
#        enter an integer that defines the cell type as promted
#        press keyboard key to move to next image in image sequence
#
#DESCRIPTION - Tracks cells and draws a bounding around each cell
#              provides number of cells in each image as an output to terminal 
#              keeps a running trajectory of the movement of cells
#
#---------------------------PACKAGES----------------------------------------------------------------
import numpy as np
import cv2
from matplotlib import pyplot as plt
import imutils
import os
#---------------------------------------------------------------------------------------------------

#datasets need different pre processing
#DIC-C2DH-HeLa-----> dataset = 1
#Fluo-N2DL-HeLa----> dataset = 2
#PhC-C2DL-PSC -----> dataset = 3

#set a default filepath
file_path = 'cell_Images/PhC-C2DL-PSC/Sequence 1'
dataset = 3

#get folder path through user input
#<-------------------------------------uncomment these lines for user defined file and dataset choice
#print("Enter image folder pathway, e.g \"cell_Images/PhC-C2DL-PSC/Sequence\" 1:") 
#file_path = input()
#print("Define cell type as an integer (large = 1, medium = 2, small = 3")
#dataset = int(input())
#----------------------------------------------------------------------------------------------------

#open first image in file to get size for the cell tracking pathway map
#assumes first file in folder is called "t000.tif"
(h,w,d) = cv2.imread(file_path+"/t000.tif").shape
pathway_mask = np.zeros((h,w,3), np.uint8)

#loop to open all the images in the given filepath and perform tracking on them
directory = os.fsencode(file_path)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #if filename.endswith(".tif"):
    #    print(file_path+"/"+filename)
    img = cv2.imread(str(file_path+"/"+filename))
    #cv2.imshow("image", img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    #define elipse based kernel as base
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  (15,15))
    
    #different preproccessing techniques for different cells
    if(dataset == 3):
        tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
        gray = cv2.cvtColor(tophat, cv2.COLOR_BGR2GRAY)
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #gaussian blur and then otsu thresholding to reduce grey noise in the cells and improve watershed segmentation
    #results
    blur = cv2.GaussianBlur(gray,(3,3),0)
    ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    #get rid of artifacts that are too small to be cells
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  (5,5))
    thresh = cv2.erode(thresh,kernel,iterations=1)
    thresh = cv2.dilate(thresh,kernel,iterations=1)
    thresh = cv2.dilate(thresh,kernel,iterations=1)
    thresh = cv2.erode(thresh,kernel,iterations=1)

    #on the PhC-C2DL-PSC dataset doing another erosion allows better sepperation of close together cells
    if dataset == 3:
        thresh = cv2.erode(thresh,kernel,iterations=1)

    #perform watershedding segmentation
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  (3,3))
    background = cv2.dilate(thresh,kernel,iterations=1)
    uncertain_area = cv2.subtract(background,thresh)

    ret, markers = cv2.connectedComponents(thresh)

    markers = markers+1
    markers[uncertain_area==255] = 0
    
    markers = cv2.watershed(img,markers)
    img[markers == -1] = [255,0,0]

    unique_markers = np.unique(markers)
    
    #print the number of cells in the image to terminal output
    #need to subtract 2 to get number of cells since background is counted twice
    print("number of cells in image("+filename+"): "+ str(len(unique_markers)-2))

    #iterate over segmented markers and draw bounding boxes for each cell
    #uses the centroid of the bound to show cell trajectory on a sepperate mask
    #TODO: find mitosis events 
    #
    for m in unique_markers:
        if m == -1 or m == 1:
            continue
        mask = np.zeros(gray.shape, dtype="uint8")
        mask[markers == m] = 255
        #find contours
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        c = max(contours, key=cv2.contourArea)
        #create bounding box for cell
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #show centroid of the cells bounding box on the pathway image
        pathway_mask[int(y+h/2),int(x+w/2)] = (255,0,0)

    #show the results on current image
    #press any key to move to next image
    #shows an ongoing cell trajectory
    cv2.imshow("thresh", thresh)
    cv2.imshow("img", img)
    cv2.imshow("pathways", pathway_mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


number of cells in image(t000.tif): 85
number of cells in image(t001.tif): 85
number of cells in image(t002.tif): 84
number of cells in image(t003.tif): 88
number of cells in image(t004.tif): 89
number of cells in image(t005.tif): 86
number of cells in image(t006.tif): 88
number of cells in image(t007.tif): 87
number of cells in image(t008.tif): 85
number of cells in image(t009.tif): 86
number of cells in image(t010.tif): 85
number of cells in image(t011.tif): 84
number of cells in image(t012.tif): 86
number of cells in image(t013.tif): 88
number of cells in image(t014.tif): 87
number of cells in image(t015.tif): 89
number of cells in image(t016.tif): 86
number of cells in image(t017.tif): 88
number of cells in image(t018.tif): 87
number of cells in image(t019.tif): 86
number of cells in image(t020.tif): 83
number of cells in image(t021.tif): 82
number of cells in image(t022.tif): 82
number of cells in image(t023.tif): 86
number of cells in image(t024.tif): 87
number of cells in image(